<a href="https://colab.research.google.com/github/pannxe/reg-kku-automator-improved/blob/main/Reg_KKU_Automate_Improve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reg KKU Automate Improve

**Reg KKU Automate Improve** is an improved version of Reg KKU Automate with some additional features.

## Features

- [ ] Range-based randomize scoring.
- [ ] Cut-off-defined reversed scoring for class scoring.

## Disclaimers

- This software is a proof-of-concept, written for educational purpose only.

- We hold NO responsibility and shall NOT be liable for any claim or damage connected to this software.

- DO NOT ACTUALLY USE THIS SOFTWARE TO AUTOMATICALLY FILL YOUR DUTY OF CLASS/TEACHER FEEDBACK FORMS 😉. Please rememeber that your feedback is important for improving the quality of education 🤫.

# Documentation

โปรแกรมจะทำการโหวต อาจารย์/วิชา (ตาม Selected) ที่เหลืออยู่ทั้งหมดโดย ~จะเลือก คะแนน **เท่ากับ Chioce ที่เลือก ทั้งหมด**~ จะสุ่มจากช่วงคะแนนที่เลือกเท่า ๆ กัน

## วิธีใช้ Program
1. เลือกหัวข้อ File แล้วทำการ Copy a in drive ก่อน จึงจะเริ่มช้งานได้
2. กรอกและเลือกข้อมูลตามที่ให้กรอกก่อน ด้านล่าง
3. กดที่ Runtime  ด้านบน และเลือก **Run all**

---
** ID** คือ รหัสนักศึกษาที่ใช้เข้า regkku แบบมีขีดเช่น 583070###-#

**Password** คือ รหัสผ่านของ regkku

---
### Selected
* ~'class' สำหรับประเมินวิชา~ Work In Progess
* 'professor' สำหรับประเมินอาจารย์

---
### Chioce คะแนน 1-5
* หากมี ตัวเลือก *ไม่ได้เรียนกับอาจารย์*  = 6
* ให้ทำการประเมินอาจารย์ที่เรียนด้วยให้เสร็จก่อนด้วยมือ และใช้โปรแกรมโดยเลือก 

# Setup

Fill in these information in order to automate your task.

## 1. Authentication Information

Provide your authentication information of reg.kku.ac.th.

> Please note that your information is stored in memory in cloud server assigned to your Google account **UNENCRYPTED**. Use this software at your own risk. **YOU HAVE BEEN WARNNED**.




In [ ]:
ID = '000000000-0' #@param {type:"string"}
password = '000000' #@param {type:"string"}

regkkuAuth = {
    'id': ID,
    'password': password
}

## 1. Select Mode

Automating `professor` or `class`? Select one and fill-in mode-specific parameters.


In [ ]:
mode = "professor" #@param ["professor", "class"] {allow-input: true}

# 2.1. Professor-specific Parameters



In [ ]:
#@title กรุณากรอกข้อมูลด้านล่าง { run: "auto", display-mode: "form" }
minScore = 4 #@param {type:"slider", min:1, max:5, step:1}
maxScore = 5 #@param {type:"slider", min:1, max:5, step:1}
startReverse = 27 #@param {type:"integer"}
lastReverse = 34 #@param {type:"integer"}

reverseScore = {
    'start' : startReverse,
    'last' : lastReverse
}

# ระบบทำงาน
ไม่ต้องสนใจ มันจะหมุนวนๆไปเรื่อยๆ เป็นการทำงานของโปรแกรม รอจนมันหมุนจนเสร็จทุกช่อง

---
ทำงานด้วย Python - Selenium




In [ ]:
# Install
print('[ SETUP ]\tDownloading dependencies...')
!pip -q install selenium

!apt-get update > /dev/null
!apt-get install -y chromium-chromedriver --fix-missing > /dev/null
!cp /usr/lib/chromium-browser/chromedriver /usr/bin > /dev/null


import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

print('[ SETUP ]\t--> Done')

In [ ]:
# Setup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

print('[ SETUP ]\tSetting up Selenium options... ', end="")
driver = webdriver.Chrome('chromedriver',options=chrome_options)
wait = WebDriverWait(driver, 10)

print('done')

In [ ]:
from time import sleep
from datetime import datetime

driver.get('https://reg.kku.ac.th/registrar/login.asp')
assert 'มหาวิทยาลัยขอนแก่น' in driver.title

# login
# username <input style="width:100px;" class="normaldetail" type="TEXT" name="f_uid" size="15" value="">
id_box = driver.find_element_by_name('f_uid')
id_box.send_keys(regkku['id'])

# pass <input type="PASSWORD" style="width:160;" name="f_pwd" size="15">
pass_box = driver.find_element_by_name('f_pwd')
pass_box.send_keys(regkku['password'])
print('[ LOGIN ] Try logging in...', end="")
pass_box.submit()
print('success')

# //*[@id="menu"]/ul/li[6]/font/a
# //*[@id="menu"]/ul/li[5]/a

selector = {
    'professor': [
        "//a[contains(@href, 'evaluate.asp?')]",
        "//a[contains(@href, 'evaluate_form.asp?')]",
        "//*[@id='menu']/ul/li[7]/font/a",
        "evaluate.asp"
    ],
    'class': [
        "//a[contains(@href, 'evaluateclass.asp?')]",
        "//a[contains(@href, 'evaluate_Class_form.asp?')]",
        "//*[@id='menu']/ul/li[6]/a",
        "evaluateclass.asp"
    ]
}

try:
  alert = wait.until(ec.alert_is_present()).dismiss()
  print(f"[ WEB ]\tAccessing : {driver.current_url}")
  print('[ WEB ]\tAlert found. Dismissed')
except:
  print('[ ERROR ]\tNo alert was found')


try:
  # evaluate = driver.find_element_by_xpath(selector[selected][2])
  # evaluate.click()
  print(f'[ WEB ]\tAccessing evaluation page : {driver.current_url} ... ', end="")
  url = 'https://reg.kku.ac.th/registrar/' + selector[selected][3]
  driver.get(url)
  print("done")
except:
  print(f"no {selected} to be evaluated")
  

In [ ]:
import random

count = 0

while(True):
    try:
      evaluate_form = driver.find_element_by_xpath(selector[selected][1])
      evaluate_form.click()
    except:
        break

    sleep(1)

    selected_choice = f"//input[@type='radio'][@valueh='{choice}']"

    choices = driver.find_elements_by_xpath(selected_choice)

    if choices == []:
        choices = driver.find_elements_by_xpath(
            "//input[@type='radio'][@value='5']")

    for c in choices:
        c.click()

    submit = driver.find_element_by_name('submit')
    submit.click()

    count += 1
    print('Completed: ', count)
    sleep(1)

driver.quit()
print(f'{selected} was finished')

In [ ]:

print(possible_choices)